## 2. Получение предсказаний на тестовой выборке и создание файла для kaggle.

Считаем данные из яндекс-диска

In [1]:
import os
import pandas as pd
path = r'/Users/romanova/Downloads/kaggle_data' 
data = {}

Data = []
Target = []

i=0
for dir_entry in sorted(os.listdir(path)):
    dir_entry_path = os.path.join(path, dir_entry)
    if os.path.isfile(dir_entry_path):
        i+=1
        with open(dir_entry_path, 'r') as my_file:
#            print(dir_entry_path.split("/")[-1])
            try:
                df = pd.read_csv(my_file, delimiter=';')
                if df.shape[1] == 5:
                    Data.append(df)
                    Target.append(dir_entry_path.split("/")[-1])
            except UnicodeDecodeError:
#                print(dir_entry_path.split("/")[-1])
                pass

for dir_entry in sorted(os.listdir(path)):
    dir_entry_path = os.path.join(path, dir_entry)
    if os.path.isfile(dir_entry_path):
        i+=1
        with open(dir_entry_path, 'r') as my_file:
#            print(dir_entry_path.split("/")[-1])
            try:
                df = pd.read_csv(my_file, delimiter=',')
                if df.shape[1] == 5:
                    Data.append(df)
                    Target.append(dir_entry_path.split("/")[-1])
            except UnicodeDecodeError:
#                print(dir_entry_path.split("/")[-1])
                pass

Переведём типы движений в числа

In [2]:
classes = {'тояни' : 0, 'месте' : 0, 'одьб' : 1, 'аг' : 1, 'ег' : 2, 'елоси' : 3, 'естн' : 4, 'одъ' : 4, 
           'втомо' : 5, 'ашин' : 5, 'метро' : 6, 'лектро' : 7, 'амок' : 8}

answers = {'стояние' : 0, 'ходьба' : 1, 'бег' : 2, 'велосипед' : 3, 'лестница' : 4, 
           'автомобиль' : 5, 'метро' : 6, 'электросамокат' : 7, 'самокат' : 8}

# стояние, ходьба, велосипед, лестница, бег, подъем, подъём, автомобиль, электросамокат, машина, метро

In [3]:
def get_action(name):
    for act in classes:
        if name.lower().find(act) != -1:
            return classes[act]
    return -1

Обработаем данные и извлечём признаки для создания модели

In [4]:
def rotation_matrix(vector):
    b = np.array([0, 0, -1])
    # Нормируем
    a = (vector / np.linalg.norm(vector)).reshape(3)
    v = np.cross(a, b) # Вычисляем векторное произведение
    
    if any(v):
        c = a * b
        s = np.linalg.norm(v)
        # Преобразование к матричной форме     
        kmat = np.array([ [0, -v[2], v[1]],
                          [v[2], 0, -v[0]],
                          [-v[1], v[0], 0] ])

        return np.eye(3) + kmat + kmat @ kmat * ((1 - c) / (s ** 2))

    else:
        return numpy.eye(3)
    
def rotate(df, mat):
    new_x = []
    new_y = []
    new_z = []
    
    for elem in zip(df.gFx.tolist(), df.gFy.tolist(), df.gFz.tolist()):
        res = mat @ elem
        new_x.append(res[0])
        new_y.append(res[1])
        new_z.append(res[2])
        
    return new_x, new_y, new_z

def integrate(Acc_arr):
    sm_arr = np.cumsum(Acc_arr).tolist()
    return sm_arr, sm_arr[-1]

In [14]:
import pandas as pd
from matplotlib import pylab as plt
from datetime import datetime
%pylab inline

def process_data(df, comma=True, cut=True, name = "", a=-1, b=-1):

# приводим данные с Iphone к нужному формату
    if comma is True:
        df = df.applymap(lambda x: str(x).replace(',','.'))

        try:
            df['gFx'] = df['gFx'].astype(float)
            df['gFy'] = df['gFy'].astype(float)
            df['gFz'] = df['gFz'].astype(float)
        except KeyError:
            return [-1, -1, -1]
        
# средняя частота сбора данных
    Freq = 0.0
    try:
        df[df.columns[0]] = df[df.columns[0]].astype(float)
        Time = (df[df.columns[0]][df.shape[0]-2] - df[df.columns[0]][1])
        Freq = round( (df.shape[0] - 2) / Time )
    except:
        Time = 20
        Freq = round( (df.shape[0] - 2) / Time )
        
# обрезаем начало и конец трека
    datFram_len = len(df) 
    toCut = round(datFram_len * 0.25)
    if a == -1:
        st = toCut
    else:
        st = a
            
    if b == -1:
        end = -toCut
    else:
        end = b
        
    if cut == False:
        st = 0
        end = -1
    

#    print(name)

#     fig, ((ax1, ax2)) = plt.subplots(nrows=1, ncols=2, figsize=(15,6))
        
########################################################
# amplitude
########################################################

# вычисляем модуль (норму) ускорения и рисуем его
    Acc = (df.gFx[st:end] ** 2 + df.gFy[st:end] ** 2 + df.gFz[st:end] ** 2) ** 0.5
#     ax2.plot(Acc)
#     plt.show()

# можно различить многие действия по амплитуде ускорения
    amplitude = Acc.max() - Acc.min()
#    print(' АМПЛИТУДА:', Acc.max()-Acc.min())
#    print()
    
            
########################################################
# преобразование Фурье
########################################################

    Fs = 250 #частота сбора данных
    Min = 0
    Acc = (df.gFx[st:end] ** 2 + df.gFy[st:end] ** 2 + df.gFz[st:end] ** 2) ** 0.5

    y = Acc.values
    if len(y) == 0:
        hesitation = -1
    else:
        n = len(y) # length of the signal
        k = np.arange(n)
        T = n/Fs
        frq = k/T # two sides frequency range
        frq = frq[:len(frq)//2] # one side frequency range

        Y = np.fft.fft(y)/n # dft and normalization
        Y = Y[1:n//2]

        yabs = abs(Y)
        xmax = frq[numpy.argmax(yabs)]
        ymax = yabs.max()

        hesitation = xmax
        if hesitation == 0:
            hesitation = -1
    
#    рисование
#    ax3.plot(frq,yabs) # plotting the spectrum
#    ax3.set_xlim([0,30])
#    ax3.set_ylim([Min,0.1])

########################################################

########################################################
# Вращаем координаты, пытаемся определить Speed и модуль перемещения
    move = []
    try:
        acc_mean = [df.gFx.mean(), df.gFy.mean(), df.gFz.mean()]
        r_mat = rotation_matrix(acc_mean)
        gx, gy, gz = rotate(df, r_mat)
        df.gFx = pd.Series(gx)
        df.gFy = pd.Series(gy)
        df.gFz = pd.Series(gz)
        
        gx = [elem / Freq ** 2 for elem in gx]
        gy = [elem / Freq ** 2 for elem in gy]
        gz = [(elem + 1) / Freq ** 2 for elem in gz]
        
        speed_arrx, speedX = integrate(gx)
        moveX, _ = integrate(speed_arrx)
        
        speed_arry, speedY = integrate(gy)
        moveY, _ = integrate(speed_arry)
        
        speed_arrz, speedZ = integrate(gz)
        moveZ, _ = integrate(speed_arrz)
        
        speed = (speedX ** 2 + speedY ** 2 + speedZ ** 2) ** 0.5
        move = [moveX[-1], moveY[-1], moveZ[-1]]
    except:
        speed = -1
########################################################     

########################################################  
# Считаем дисперсию
    # среднее по модулю ускорения
    mean_a = Acc.mean()
    disp = 0.0
    for row in Acc:
        disp += (row - mean_a) ** 2
    disp /= (df.shape[0] - 1)
########################################################  

#    print('Амплитуда:', amplitude, ' xmax:', xmax, ' ymax:', ymax, 'speed:', speed, 'disp:', disp, 'move:', move)
    return [amplitude, xmax, ymax, yabs, speed, disp, move]

Information = []
for i in range (len(Data)):
    Information.append(np.zeros(7))

for df, target in zip(Data,Target):
    track_num = int(target.split('_')[1].split('.')[0]) # важный момент, что треки не по нумерации идут!
    Information[track_num] = process_data(df, name=target)

Populating the interactive namespace from numpy and matplotlib


Напишем функцию для получения предсказания (здесь используется только амплитуда, но в вашем итоговом решении будет больше признаков)

In [28]:
def get_prediction_base(info):
    ampl, xmax, ymax, yabs, speed, disp, move = info[0], info[1], info[2], info[3], info[4], info[5], info[6]
    if ampl < 1 and max(yabs) < 0.015:
        return 0 # стояние
    if ampl > 4 and disp > 0.45 and max(yabs) > 0.07:
        return 2 # бег
    if 1 <= ampl <= 5:
        if 0.2 < xmax < 3.1 and abs(move[1]) > max(abs(move[0]), abs(move[2])):
            return 4 # лестница
        elif 0.9 < xmax < 2.1:
            return 1 # ходьба
        if disp < 0.25 and speed > 0.03:
            return 3 # велосипед
    return 1

In [29]:
def get_prediction_advanced(info):
    ampl, xmax, ymax, yabs, speed, disp, move = info[0], info[1], info[2], info[3], info[4], info[5], info[6]
    if speed > 0.3:
        return 8
    if speed > 0.1:
        return 5 # автомобиль 
    if ampl < 1.5:
        if 0.15 < ampl < 1 and disp < 0.0025:
            if 0.4 < ampl < 0.6 and 0.001 < disp < 0.002:
                return 5 # автомобиль
            return 6 # метро
        if 0.4 < ampl and disp < 0.03:
            return 5 # автомобиль
    return 7 # электросамокат

In [30]:
len(Data), len(Information), len(Target)

(146, 146, 146)

In [31]:
Result = []

count = 0
for i in range (len(Information)):
    if not ((math.isnan(Information[i][0]) or Information[i][0]==-1) and 
            (math.isnan(Information[i][1]) or Information[i][1]==-1) and 
            (math.isnan(Information[i][2]) or Information[i][2]==-1) and
            (math.isnan(Information[i][3]) or Information[i][3]==-1) and
            (math.isnan(Information[i][4]) or Information[i][4]==-1) ):
        count += 1
        Result.append(Information[i])
    else:
        Result.append([2, 2, 2, 0.1, 0.1])
len(Result), count

(146, 146)

Создадим таблицу, содержащую номер трека и предсказание.

In [32]:
tracks_levels = pd.read_csv('/Users/romanova/Downloads/tracks_levels_3.csv')

In [33]:
prediction = [1] * len(Result)
for i in range(len(tracks_levels)):
    if tracks_levels['level'][i] == 'advanced':
        prediction[i] = get_prediction_advanced(Result[i])
    else:
        prediction[i] = get_prediction_base(Result[i])

Res = pd.DataFrame({'track_num' : np.arange(len(Data)), 'action' : prediction})
Res.head(10)

,track_num,action
0,0,3
1,1,1
2,2,4
3,3,2
4,4,4
5,5,3
6,6,1
7,7,2
8,8,1
9,9,0


In [34]:
Res['action'].value_counts()

1    40
3    34
0    26
2    18
4    17
5     6
7     2
6     2
8     1
Name: action, dtype: int64

In [35]:
Res.to_csv("Romanova_Anastasia_3.csv", index=False)